In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/study/16715robotsim/project/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Test
using Plots

In [136]:
const g = 9.8
const m = 1
μ = 0.5

w = 1.0 # width of the box
h = 0.8 # height of the box
M = [m 0 0; 0 m 0; 0 0 m*((h/2)^2+(w/2)^2)/3]

# set of contact modes
modes = [0 0 0 0; # both free
        1 0 0 0; # sticking
        1 0 -1 0; # left-slide
        1 0 1 0; # right-slide
        0 1 0 0;
        0 1 0 -1;
        0 1 0 1;
        1 1 0 0;
        1 1 -1 -1;
        1 1 1 1]

q0 = [0;0.5;0] # x y θ
dq0 = [0;0;0]
x0 = [q0;dq0]
initial_mode = [0 0 0 0]

n_contacts = 2


2

In [137]:
function R_2D(θ)
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return R
end

R_2D (generic function with 1 method)

In [138]:
# constraints/contacts
function compute_a(q)
    y = q[2]
    θ = q[3]
    a1 = y - 0.5*h*cos(θ) - 0.5*w*sin(θ)
    a2 = y - 0.5*h*cos(θ) + 0.5*w*sin(θ)
    return [a1; a2]
end

function compute_a_t(q)
    p1 = R_2D(q[3])*[-w/2; -h/2] 
    p2 = R_2D(q[3])*[w/2; -h/2] 
    at1 = p1[1] + q[1]
    at2 = p2[1] + q[1]
    return [at1; at2]
end

# constraints jacobian
function compute_A(q)
    θ = q[3]
    A = [0 1 0.5*h*sin(θ)-0.5*w*cos(θ); 0 1 0.5*h*sin(θ)+0.5*w*cos(θ)]
    return A
end

function compute_dA(q, dq)
    dA = reshape(ForwardDiff.jacobian(compute_A, q)*dq, n_contacts, 3)
    return dA
end

function compute_A_tangent(q)
    A_t = ForwardDiff.jacobian(compute_a_t,q)
    return A_t
end

function compute_dA_tangent(q, dq)
    dA_t = reshape(ForwardDiff.jacobian(compute_A_tangent, q)*dq, n_contacts, 3)
    return dA_t
end

compute_dA_tangent (generic function with 1 method)

In [139]:
println(compute_a(q0))
println(compute_A(q0))
println(compute_dA(q0, dq0))
println(compute_A_tangent(q0))
println(compute_dA_tangent(q0, dq0))

[0.09999999999999998, 0.09999999999999998]
[0.0 1.0 -0.5; 0.0 1.0 0.5]
[0.0 0.0 0.0; 0.0 0.0 0.0]
[1.0 0.0 0.4; 1.0 0.0 0.4]
[0.0 0.0 0.0; 0.0 0.0 0.0]


In [140]:
function contact_mode_constraints(x, contactMode)
    
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    A_all = zeros(0,3)
    A_all_f = zeros(0,3)
    dA_all = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            A_all_f = [A_all_f; A[k,:]'; A_t[k,:]']
            dA_all = [dA_all; dA[k,:]'; dA_t[k,:]']
            A_all = [A_all; A[k,:]'; A_t[k,:]']
        else
            A_all_f = [A_all_f; A[k,:]'-ss*μ*A_t[k,:]']
            dA_all = [dA_all; dA[k,:]']
            A_all = [A_all; A[k,:]']
        end
    end
    
    return A_all_f, A_all, dA_all
end

contact_mode_constraints (generic function with 1 method)

In [141]:
contact_mode_constraints([0;0;0;0;0;0], [1 1 1 1])

([-0.5 1.0 -0.7; -0.5 1.0 0.3], [0.0 1.0 -0.5; 0.0 1.0 0.5], [0.0 0.0 0.0; 0.0 0.0 0.0])

In [142]:
function contact_force_constraints(λ, contactMode)
    # c > 0
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    ss_active = ss_mode[cs_mode]
    
    i = 1
    ic = 1
    c = zeros(sum(ss_active.==0)*3 + sum(ss_active.!=0))
    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            c[ic:ic+2] = [-λ[i]; -μ*λ[i] - λ[i+1]; -μ*λ[i] + λ[i+1]]
            ic += 3
            i += 2
        else
            c[ic] = -λ[i]
            i += 1
            ic += 1
        end
    end
    return c
end

contact_force_constraints (generic function with 1 method)

In [143]:
println(contact_force_constraints([-10;-8;-10;-6], [1 1 0 0]))

[10.0, 13.0, -3.0, 10.0, 11.0, -1.0]


In [144]:
function sliding_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A[k,:]'; A_t[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]'; dA_t[k,:]']
        else
            AA_eq = [AA_eq; A[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]']
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

sliding_velocity_constraints (generic function with 1 method)

In [145]:
function solveEOM(x, contactMode)
    # contactMode: bool vector, indicates which constraints are active
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = [0;0;0]
    
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    #z = blockMat\b
    #println(blockMat)
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end
    
    ddq = z[1:3]
    if (sum(contactMode[1:n_contacts])>=1)
        λ = z[4:end]
    else
        λ = []
    end
    
    return ddq, λ
end

solveEOM (generic function with 1 method)

In [146]:
println(solveEOM([0;1;0;0;0;0],[0 0 0 0]))
println(solveEOM([0;0.5;0;0;0;0],[1 1 0 0]))
println(solveEOM([0;0.5;0;-1;0;0],[1 1 -1 -1]))

([0.0, -9.8, 0.0], Any[])
([1.5338137800938238e-15, -5.440092820663239e-16, -2.475581045774359e-18], [-4.900000000000005, -4.2815550134741735e-14, -4.900000000000002, 3.965841918331751e-14])
([4.9, 0.0, -0.0], [-6.860000000000001, -2.9399999999999995])


In [147]:
function computeResetMap(x, contactMode)
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    if rank(blockMat) < 3+c
        z = pinv(blockMat)*[M*dq; zeros(c)]
    else
        z = blockMat\[M*dq; zeros(c)]
    end
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    return dq_p, p_hat
end

computeResetMap (generic function with 1 method)

In [148]:
dq_p, p_hat = computeResetMap([0;0;0;1;-1;0],[1 1 1 1])

([0.5, 0.0, -0.0], [-0.30000000000000004, -0.7])

In [149]:
function compute_FA(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end

    max_cons = 0
    
    for kk = 1:size(possibleModes, 1)      
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, dA_separate = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        ddq, λ = solveEOM(x, m)
        
        c_λ = contact_force_constraints(λ, m)
        
        if all(c_λ.>=0)
        
            As_eq, As_geq, dAs_eq, dAs_geq = sliding_velocity_constraints(x, m)

            sep_vel_cond = ((A_separate*dq).>0) .| ((A_separate*ddq .+ dA_separate*dq).>=0)
            maintain_vel_cond = all(abs.(As_eq*dq).<1e-6) & all((As_geq*dq).>1e-6)

            if ~maintain_vel_cond
                if any((As_geq*dq).<1e-6)
                    maintain_vel_cond = all(abs.(dAs_eq*dq + As_eq*ddq).<1e-6) & all((dAs_geq*dq + As_geq*ddq).>0)
                end
            end

            if all(c_λ.>=0) && all(sep_vel_cond) && maintain_vel_cond
                if sum(m[1:n_contacts]) > max_cons
                    contactMode = m
                    max_cons = sum(m[1:n_contacts])
                end
            end
        end
    end
    
    return contactMode
end

compute_FA (generic function with 1 method)

In [150]:
println(compute_FA([0;0.5;0;1;0;0]))
println(compute_FA([0;0.5;0;-1;0;0]))
println(compute_FA([0;0.5;0;0;0;0]))
println(compute_FA([0;1;0;0;0;0]))

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [151]:
function compute_IV(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end
    
    max_cons = 0
    for kk = 1:size(possibleModes, 1)
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        dq_p, p_hat = computeResetMap(x, m)
        
        c_p_hat = contact_force_constraints(p_hat, m)
        As_eq, As_geq, _, _ = sliding_velocity_constraints(x, m)
        
        if all(c_p_hat.>=0) && all(A_separate*dq_p.>=0) && all(abs.(As_eq*dq_p).<1e-6) && all(As_geq*dq_p.>0)
             if sum(m[1:n_contacts]) > max_cons
                contactMode = m
                max_cons = sum(m[1:n_contacts])
            end
        end
    end
    
    return contactMode
end

compute_IV (generic function with 1 method)

In [152]:
println(compute_IV([0;0.5;0;0;-1;0]))
println(compute_IV([0;0.5;0;-1;-1;0]))
println(compute_IV([0;0.5;0;1;-1;0]))
println(compute_IV([0;0.5;0;0;0;0.5]))

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [153]:
function guard_conditions(x, contactMode)
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    a[contactMode[1:n_contacts] .== 1] .= 0.0
    
    v_all = zeros(n_contacts)
    _, As_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    v_all[1:size(As_geq,1)] = -As_geq*dq
    
    ddq, λ = solveEOM(x, contactMode)
    c_λ = contact_force_constraints(λ, contactMode)
    c_λ_all = zeros(3*n_contacts)
    c_λ_all[1:length(c_λ)] = c_λ
    
    c = [a; v_all; c_λ_all]
    
    dir = [-ones(Int,length(a)); ones(Int,length(v_all)); ones(Int,length(c_λ_all))]
    
    return c, dir
end

guard_conditions (generic function with 1 method)

In [154]:
guard_conditions([0;0.5;0;0;0;0],[1 1 0 0])

([0.0, 0.0, 0.0, 0.0, 4.900000000000005, 2.450000000000045, 2.4499999999999598, 4.900000000000002, 2.4499999999999615, 2.4500000000000406], [-1, -1, 1, 1, 1, 1, 1, 1, 1, 1])

In [155]:
function dynamics!(dx, x, m, t)
    # m: vector n, contact mode
    q = x[1:3]
    dq = x[4:6]
    ddq, λ = solveEOM(x, m)
    dx .= [dq; ddq]
end

dynamics! (generic function with 1 method)

In [164]:
function conditions(out, x, t, integrator)
    contactMode = integrator.p
    c, dir = guard_conditions(x, contactMode)
    out .= c
end

function affect!(integrator, idx)
    contactMode = integrator.p
    x = integrator.u
    c, dir = guard_conditions(x, contactMode)
    
    # only consider upcrossing forces and constraints values(FA comp)
    # forces
    if dir[idx] > 0
        new_contactMode = compute_FA(x)
        integrator.p .= reshape(new_contactMode,size(integrator.p))
        println("New mode from FA: ", new_contactMode)
    end
    # constraints
#     if dir[idx] < 0
#         new_contactMode = contactMode
#         new_contactMode[idx] = false
#     end
end

function affect_neg!(integrator, idx)
    contactMode = integrator.p
    x = integrator.u
    c, dir = guard_conditions(x, contactMode)
    
    # only consider down crossing constraint value(IV comp)
    if dir[idx] < 0
        new_contactMode = compute_IV(x)
        dq_p, p_hat = computeResetMap(x, new_contactMode)
        integrator.u .= [x[1:3]; dq_p]
        integrator.p .= reshape(new_contactMode,size(integrator.p))
        println("New mode from IV: ", new_contactMode)

    end
end

affect_neg! (generic function with 1 method)

In [165]:
tspan = (0.0, 5.0)
callback_length = 5*n_contacts

x0 = [0;sin(0.3)*w/2+1.5;0.3;0;0;0]
initial_mode = [0 0 0 0]
prob = ODEProblem(dynamics!, x0, tspan, initial_mode)
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15)

New mode from IV: [1, 0, 0, 0]
New mode from IV: [1, 1, 1, 1]
New mode from FA: [1, 1, 0, 0]


retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 151-element Vector{Float64}:
 0.0
 0.00030169624850661715
 0.0007998652732074697
 0.0015672035404869365
 0.0026907095618956133
 0.004306495077333731
 0.006586337915122843
 0.009747695149637954
 0.014062300172284856
 0.019866652163287465
 0.02757546702282971
 0.03769842675659535
 0.05086067193706908
 ⋮
 0.6084862946412378
 0.6260768845699644
 0.6489289112007489
 0.6779444169404357
 0.6809471963273143
 0.6809471963273143
 0.7170410855705089
 1.0779799780024555
 1.286839028016251
 3.3754295281542044
 4.273556526320733
 5.0
u: 151-element Vector{Vector{Float64}}:
 [0.0, 1.6477601033306697, 0.3, 0.0, 0.0, 0.0]
 [0.0, 1.6477596573296005, 0.3, 0.0, -0.002956623235364847, 0.0]
 [0.0, 1.6477569683868387, 0.3, 0.0, -0.007838679677433201, 0.0]
 [0.0, 1.6477480683086767, 0.3, 0.0, -0.015358594696771973, 0.0]
 [0.0, 1.6477246277327318, 0.3, 0.0, -0.026368953706577002, 0.0]
 [0.0, 1.6476692284213992, 0.3, 0.0, -0.042203651

In [161]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

boxshape (generic function with 1 method)

In [167]:
# animation 
n = length(sol.t)
x = zeros(n)
y = zeros(n)
θ = zeros(n)
for k = 1:n
    x[k] = sol.u[k][1]
    y[k] = sol.u[k][2]
    θ[k] = sol.u[k][3]
end
    
anim = @animate for i ∈ 1:n
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-3,3), ylims=(-0.5,3))
    plot!(boxshape([x[i],y[i],θ[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
    
end

Animation("/tmp/jl_deaRYv", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000142.png", "000143.png", "000144.png", "000145.png", "000146.png", "000147.png", "000148.png", "000149.png", "000150.png", "000151.png"])

In [170]:
gif(anim, "anim.gif", fps = 1000);

┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/anim.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
